## Imports

In [94]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import os
import pickle
from dateutil.parser import parse
from datetime import datetime
sns.set_context('notebook')

## Constants

In [2]:
DATA_PATH = './data/food-inspections.csv'
#DEFAULT_ENCODING = 'UTF8'
#DEFAULT_COMPRESSION = 'gzip'

## Data preparation

### Import data

In [185]:
df = pd.read_csv(DATA_PATH, na_values=[0, 0.0])
df.head(3)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
0,2315925,EXOS WORKS,EXOS WORKS,2658207.0,Restaurant,Risk 1 (High),401 N MICHIGAN AVE,CHICAGO,IL,60611.0,...,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.889354,-87.624102,"{'longitude': '41.88935425192819', 'latitude':...",NaN,NaN,NaN,NaN,NaN
1,2315916,INTRINSIC CHARTER SCHOOL,INTRINSIC CHARTER SCHOOL,3305837.0,School,Risk 1 (High),4540 W BELMONT AVE,CHICAGO,IL,60641.0,...,Fail,59. PREVIOUS PRIORITY FOUNDATION VIOLATION COR...,41.939016,-87.741148,"{'longitude': '41.93901560699069', 'latitude':...",NaN,NaN,NaN,NaN,NaN
2,2315931,STARBUCKS COFFEE #49894,STARBUCKS,2621807.0,Restaurant,Risk 2 (Medium),5601 W LAWRENCE AVE,CHICAGO,IL,60630.0,...,Pass w/ Conditions,"48. WAREWASHING FACILITIES: INSTALLED, MAINTAI...",41.967671,-87.767596,"{'longitude': '41.967671015100265', 'latitude'...",NaN,NaN,NaN,NaN,NaN


In [186]:
# Remove duplicates
df.drop_duplicates(subset=['Inspection ID'], inplace=True)
df = df.set_index('Inspection ID')
df.head(3)

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
Inspection ID,,,,,,,,,,,,,,,,,,,,,
2315925,EXOS WORKS,EXOS WORKS,2658207.0,Restaurant,Risk 1 (High),401 N MICHIGAN AVE,CHICAGO,IL,60611.0,2019-10-16T00:00:00.000,...,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.889354,-87.624102,"{'longitude': '41.88935425192819', 'latitude':...",NaN,NaN,NaN,NaN,NaN
2315916,INTRINSIC CHARTER SCHOOL,INTRINSIC CHARTER SCHOOL,3305837.0,School,Risk 1 (High),4540 W BELMONT AVE,CHICAGO,IL,60641.0,2019-10-16T00:00:00.000,...,Fail,59. PREVIOUS PRIORITY FOUNDATION VIOLATION COR...,41.939016,-87.741148,"{'longitude': '41.93901560699069', 'latitude':...",NaN,NaN,NaN,NaN,NaN
2315931,STARBUCKS COFFEE #49894,STARBUCKS,2621807.0,Restaurant,Risk 2 (Medium),5601 W LAWRENCE AVE,CHICAGO,IL,60630.0,2019-10-16T00:00:00.000,...,Pass w/ Conditions,"48. WAREWASHING FACILITIES: INSTALLED, MAINTAI...",41.967671,-87.767596,"{'longitude': '41.967671015100265', 'latitude'...",NaN,NaN,NaN,NaN,NaN


In [187]:
# The index is now unique
df.index.is_unique

True

In [188]:
df.dtypes

DBA Name                       object
AKA Name                       object
License #                     float64
Facility Type                  object
Risk                           object
Address                        object
City                           object
State                          object
Zip                           float64
Inspection Date                object
Inspection Type                object
Results                        object
Violations                     object
Latitude                      float64
Longitude                     float64
Location                       object
Historical Wards 2003-2015    float64
Zip Codes                     float64
Community Areas               float64
Census Tracts                 float64
Wards                         float64
dtype: object

In [189]:
# Drop the columns composed of only NaN values
df.drop(['Location', 'Historical Wards 2003-2015', 'Zip Codes', 'Community Areas', 'Census Tracts', 'Wards'], axis=1, inplace=True)
df.head(3)

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
Inspection ID,,,,,,,,,,,,,,,
2315925,EXOS WORKS,EXOS WORKS,2658207.0,Restaurant,Risk 1 (High),401 N MICHIGAN AVE,CHICAGO,IL,60611.0,2019-10-16T00:00:00.000,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.889354,-87.624102
2315916,INTRINSIC CHARTER SCHOOL,INTRINSIC CHARTER SCHOOL,3305837.0,School,Risk 1 (High),4540 W BELMONT AVE,CHICAGO,IL,60641.0,2019-10-16T00:00:00.000,Canvass Re-Inspection,Fail,59. PREVIOUS PRIORITY FOUNDATION VIOLATION COR...,41.939016,-87.741148
2315931,STARBUCKS COFFEE #49894,STARBUCKS,2621807.0,Restaurant,Risk 2 (Medium),5601 W LAWRENCE AVE,CHICAGO,IL,60630.0,2019-10-16T00:00:00.000,Canvass,Pass w/ Conditions,"48. WAREWASHING FACILITIES: INSTALLED, MAINTAI...",41.967671,-87.767596


In [190]:
show_all(df['Inspection Date'].value_counts())

2013-11-14T00:00:00.000    185
2016-09-13T00:00:00.000    149
2016-11-29T00:00:00.000    145
2014-12-16T00:00:00.000    143
2013-10-01T00:00:00.000    142
2014-12-03T00:00:00.000    142
2014-12-02T00:00:00.000    140
2016-09-29T00:00:00.000    140
2016-09-26T00:00:00.000    138
2016-08-18T00:00:00.000    137
2016-09-08T00:00:00.000    137
2016-09-15T00:00:00.000    136
2016-10-04T00:00:00.000    135
2016-06-16T00:00:00.000    135
2016-09-09T00:00:00.000    134
2011-03-28T00:00:00.000    134
2016-09-30T00:00:00.000    133
2018-06-05T00:00:00.000    133
2016-08-29T00:00:00.000    133
2013-09-10T00:00:00.000    132
2016-09-16T00:00:00.000    132
2010-06-07T00:00:00.000    132
2017-11-07T00:00:00.000    132
2016-05-03T00:00:00.000    132
2013-09-09T00:00:00.000    132
2016-08-30T00:00:00.000    132
2017-03-21T00:00:00.000    132
2016-05-26T00:00:00.000    131
2011-02-25T00:00:00.000    131
2014-12-04T00:00:00.000    131
2016-10-05T00:00:00.000    130
2015-09-29T00:00:00.000    130
2013-09-

In [191]:
# Remove useless information in date
df['Inspection Date'] = df['Inspection Date'].apply(lambda x: parse(x[:10]))
df.head(3)

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
Inspection ID,,,,,,,,,,,,,,,
2315925,EXOS WORKS,EXOS WORKS,2658207.0,Restaurant,Risk 1 (High),401 N MICHIGAN AVE,CHICAGO,IL,60611.0,2019-10-16,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.889354,-87.624102
2315916,INTRINSIC CHARTER SCHOOL,INTRINSIC CHARTER SCHOOL,3305837.0,School,Risk 1 (High),4540 W BELMONT AVE,CHICAGO,IL,60641.0,2019-10-16,Canvass Re-Inspection,Fail,59. PREVIOUS PRIORITY FOUNDATION VIOLATION COR...,41.939016,-87.741148
2315931,STARBUCKS COFFEE #49894,STARBUCKS,2621807.0,Restaurant,Risk 2 (Medium),5601 W LAWRENCE AVE,CHICAGO,IL,60630.0,2019-10-16,Canvass,Pass w/ Conditions,"48. WAREWASHING FACILITIES: INSTALLED, MAINTAI...",41.967671,-87.767596


In [192]:
df.dtypes

DBA Name                   object
AKA Name                   object
License #                 float64
Facility Type              object
Risk                       object
Address                    object
City                       object
State                      object
Zip                       float64
Inspection Date    datetime64[ns]
Inspection Type            object
Results                    object
Violations                 object
Latitude                  float64
Longitude                 float64
dtype: object

In [193]:
df = df[df['Inspection Date'] < datetime(2018, 7, 1)]

In [194]:
df.sort_values('Inspection Date', ascending=False)

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
Inspection ID,,,,,,,,,,,,,,,
2182172,PEPE'S RESTAURANT,PEPE'S RESTAURANT,7056.0,Restaurant,Risk 1 (High),7026 W ARCHER AVE,CHICAGO,IL,60638.0,2018-06-29,Complaint,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.792140,-87.797669
2182154,THE GOOD LIFE SOUL FOOD RESTAURANT,THE GOOD LIFE SOUL FOOD RESTAURANT,2551347.0,Restaurant,Risk 1 (High),11142 S HALSTED ST,CHICAGO,IL,60628.0,2018-06-29,Canvass Re-Inspection,Pass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.691077,-87.642490
2182156,COLUMBUS MANOR RESIDENTIAL CARE & HOME,COLUMBUS MANOR RESIDENTIAL CARE & HOME,2418662.0,Long Term Care,Risk 1 (High),5107-5121 W JACKSON BLVD,CHICAGO,IL,60644.0,2018-06-29,Canvass Re-Inspection,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.876564,-87.753745
2182166,IZALCO RESTAURANT,IZALCO RESTAURANT,1842835.0,Restaurant,Risk 1 (High),4377 W 26TH ST,CHICAGO,IL,60623.0,2018-06-29,Canvass Re-Inspection,Pass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.844006,-87.734046
2182164,ALTGELD GARDEN LIQUOR INC,ALTGELD GARDEN LIQUOR INC,32894.0,Grocery Store,Risk 3 (Low),13118 S ELLIS AVE,CHICAGO,IL,60827.0,2018-06-29,Complaint Re-Inspection,Pass,NaN,41.656753,-87.597599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948.0,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631.0,2010-01-04,License,Fail,18. NO EVIDENCE OF RODENT OR INSECT OUTER OPEN...,NaN,NaN
52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,2010-01-04,License Re-Inspection,Pass,NaN,41.938007,-87.644755
67732,WOLCOTT'S,TROQUET,1992039.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,2010-01-04,License Re-Inspection,Pass,NaN,41.961606,-87.675967


In [195]:
df['City'].value_counts()

CHICAGO                 172001
Chicago                    285
chicago                     89
CCHICAGO                    42
SCHAUMBURG                  22
MAYWOOD                     15
ELK GROVE VILLAGE           13
CHicago                     11
EVANSTON                    10
CICERO                       9
CHESTNUT STREET              9
INACTIVE                     8
SKOKIE                       8
CHCHICAGO                    6
NILES NILES                  6
WORTH                        5
OAK PARK                     5
ELMHURST                     5
CALUMET CITY                 5
CHICAGOCHICAGO               4
CHARLES A HAYES              4
SUMMIT                       4
SCHILLER PARK                3
EAST HAZEL CREST             3
CHCICAGO                     3
CHICAGOI                     3
BRIDGEVIEW                   3
ALSIP                        3
BLUE ISLAND                  2
STREAMWOOD                   2
BANNOCKBURNDEERFIELD         2
BERWYN                       2
BEDFORD 

In [196]:
df['State'].value_counts()

IL    172716
Name: State, dtype: int64

In [197]:
# Remove rows corresponding to facilities located in New York or Wisconsin
df = df[(df['State'] != 'NY') & (df['State'] != 'WI')]

In [198]:
df['State'].value_counts()

IL    172716
Name: State, dtype: int64

In [199]:
def show_all(data):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(data)

In [200]:
def normalize_city(city):
    """
    TODO
    """
    normalized = str(city).upper()
    if ('CHICAGO' in normalized and ' ' not in normalized) or 'CHCICAGO' == normalized:
        normalized = 'CHICAGO'
    elif 'OOLYMPIA FIELDS' == normalized:
        normalized = 'OLYMPIA FIELDS'
    
    return normalized

In [201]:
# Clean city column
df.dropna(subset=['City'], inplace=True)
df = df[df['City'] != 'INACTIVE']
df['City'] = df['City'].apply(normalize_city)
show_all(df['City'].value_counts())

CHICAGO                 172447
SCHAUMBURG                  22
MAYWOOD                     16
ELK GROVE VILLAGE           13
EVANSTON                    10
CHESTNUT STREET              9
CICERO                       9
SKOKIE                       8
NILES NILES                  6
WORTH                        5
OAK PARK                     5
ELMHURST                     5
CALUMET CITY                 5
CHARLES A HAYES              4
ALSIP                        4
SUMMIT                       4
BRIDGEVIEW                   3
SCHILLER PARK                3
EAST HAZEL CREST             3
NAPERVILLE                   2
CHICAGO HEIGHTS              2
BLUE ISLAND                  2
BEDFORD PARK                 2
OLYMPIA FIELDS               2
BERWYN                       2
BANNOCKBURNDEERFIELD         2
STREAMWOOD                   2
TINLEY PARK                  1
BURNHAM                      1
BLOOMINGDALE                 1
GLENCOE                      1
OAK LAWN                     1
LAKE ZUR

In [202]:
df.head(3)

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude
Inspection ID,,,,,,,,,,,,,,,
2182172,PEPE'S RESTAURANT,PEPE'S RESTAURANT,7056.0,Restaurant,Risk 1 (High),7026 W ARCHER AVE,CHICAGO,IL,60638.0,2018-06-29,Complaint,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.792140,-87.797669
2182156,COLUMBUS MANOR RESIDENTIAL CARE & HOME,COLUMBUS MANOR RESIDENTIAL CARE & HOME,2418662.0,Long Term Care,Risk 1 (High),5107-5121 W JACKSON BLVD,CHICAGO,IL,60644.0,2018-06-29,Canvass Re-Inspection,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.876564,-87.753745
2182153,SHAN SHAAN TASTE,SHAN SHAAN TASTE,2608521.0,Restaurant,Risk 1 (High),2002 S WENTWORTH AVE,CHICAGO,IL,60616.0,2018-06-29,License,No Entry,NaN,41.855282,-87.631993


In [203]:
df['License #'] = df['License #'].dropna().astype(int)
df['Zip'] = df['Zip'].dropna().astype(int)
df.head(3)

In [174]:
df.dtypes

DBA Name                   object
AKA Name                   object
License #                 float64
Facility Type              object
Risk                       object
Address                    object
City                       object
State                      object
Zip                       float64
Inspection Date    datetime64[ns]
Inspection Type            object
Results                    object
Violations                 object
Latitude                  float64
Longitude                 float64
dtype: object